In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, RobertaTokenizer, RobertaForSequenceClassification
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_excel('/content/dataset_final.xlsx')

# Print out the column names to inspect them
print("Columns in the dataset:", df.columns)

# Replace 'Question' and 'Discipline' with your actual column names
df = df[['Question', 'Discipline']]  # Assuming 'Question' is text and 'Discipline' is the label column
# Remove rows with missing values
df = df.dropna()

# Label Encoding for the 'Discipline' column
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Discipline'])

# Train-Test Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the data to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Choose a model ('bert-base-uncased' or 'roberta-base')
model_name = 'bert-base-uncased'

if model_name.startswith('bert'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())
elif model_name.startswith('roberta'):
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].nunique())

# Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples['Question'], padding='max_length', truncation=True)

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Ensure 'Discipline' is encoded as 'label'
train_dataset = train_dataset.rename_column('label', 'labels')
test_dataset = test_dataset.rename_column('label', 'labels')

# Set format for PyTorch (ensure 'labels' is included)
target_columns = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type='torch', columns=target_columns)
test_dataset.set_format(type='torch', columns=target_columns)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)  # Print out the evaluation results for comparison purposes



Columns in the dataset: Index(['Discipline', 'Question', 'Reason you think it is faulty',
       'Which top LLM you tried ', 'Response by a top LLM '],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.097788
2,0.578100,0.021463
3,0.039600,0.007963


{'eval_loss': 0.007962885312736034, 'eval_runtime': 23.6281, 'eval_samples_per_second': 37.286, 'eval_steps_per_second': 4.698, 'epoch': 3.0}
